<h2>CShelves IGH GCS global and regional conversions</h2>

The core scripts in this notebook support the partitioning, export, and re-projection of raster data associated with research behind the Geophysical Research Letters published paper: _Where the continent ends_

- Script 1 partitions the existing global shelf converage raster (ETOPO1_World_Shelf_1km or ETOPO1_World_Shelf_NoPoles_1km) into the 12 regions as defined by the data's map projection: the Interrupted Goode Homolosine. It then exports the raster to a format of the user's choice (via the format argument) to an external raster image file outside of GRASS.

- Script 2


NOTES:
1. Because of requirements/limitations with GRASS, you must run these scripts in a Python 2.7 environment

In [32]:
# This CLASS definition is used to initialize an instance of GRASS 7.x in this domain space
class GrassBASE:
    def initGRASSEnv( self, database, loc, map ):
        gisbase=os.environ[ 'GISBASE' ] = '/Applications/GRASS-7.3.app/Contents/MacOS'
        gisdbase=database
        location=loc
        mapset=map
        sys.path.append(os.path.join(os.environ['GISBASE'], "etc", "python"))
        import grass.script as grass
        import grass.script.setup as gsetup
        gsetup.init( gisbase, gisdbase, location, mapset )
        print(grass.gisenv())

In [33]:
import sys, os

# set the database, location, and mapset for the GRASS instance to be created
gisdbase = '/Users/paulp/Documents/projects/ContinentalShelves/cshelves/data/GRASSData'    
location = 'csi_shelf_mapping_IGH'                                  
mapset = 'user'         


# initiate a GRASS instance/environment
print('')
print ('********************************************')
print ('Setting up GRASS Environment')
g = GrassBASE()
g.initGRASSEnv( gisdbase, location, mapset )


********************************************
Setting up GRASS Environment
{u'MAPSET': u'user', u'GISDBASE': u'/Users/paulp/Documents/projects/ContinentalShelves/cshelves/data/GRASSData', u'LOCATION_NAME': u'csi_shelf_mapping_IGH'}


**Script #1**

In [55]:
### SCRIPT #1
from grass.pygrass.modules.shortcuts import general as g
from grass.pygrass.modules.shortcuts import raster as r
from grass.pygrass.modules import Module

r_out_gdal=Module('r.out.gdal')

regions=['Region01_BBox','Region02_BBox','Region03_BBox','Region04_BBox','Region05_BBox',
         'Region06_BBox','Region07_BBox','Region08_BBox','Region09_BBox','Region10_BBox',
         'Region11_BBox','Region12_BBox']

shelf_rast='ETOPO1_World_Shelf_1km'

for region in regions:
    # set computational region
    g.region(vector=region)

    outrast='ETOPO1_Region'+region[6:8]+'_Shelf_1km'
    r.mapcalc( '%s = %s' % (outrast, shelf_rast), overwrite = True)

    inpt=outrast
    outpt='/Users/paulp/shelves_data/'+outrast
    r_out_gdal(input=inpt,output=outpt, format='GTiff', createopt='TFW=YES', overwrite=True)

**Script #2**

In [85]:
### Script #2

import os

rpath='/Users/paulp/shelves_data/'
src_proj=['+proj=moll +lon_0=100w +x_0=-11119487.42847 +y_0=-336410.83237',
         '+proj=moll +lon_0=30E +x_0=3335846.22854 +y_0=-336410.83237',
          '+proj=sinu +lon_0=100W +x_0=-11119487.42847 +y_0=0.0',
          '+proj=sinu +lon_0=30E +x_0=3335846.22854 +y_0=0.0',
          '+proj=sinu +lon_0=160W +x_0=-17791179.88555 +y_0=0.0',
          '+proj=sinu +lon_0=60W +x_0=-6671692.45708 +y_0=0.0 ',
          '+proj=sinu +lon_0=20E +x_0=2223897.48569 +y_0=0.0',
          '+proj=sinu +lon_0=140E +x_0=15567282.39985 +y_0=0.0',
          '+proj=moll +lon_0=160W +x_0=-17791179.88555 +y_0=336410.83237',
          '+proj=moll +lon_0=60W +x_0=-6671692.45708 +y_0=336410.83237',
          '+proj=moll +lon_0=20E +x_0=2223897.48569 +y_0=336410.83237',
          '+proj=moll +lon_0=140E +x_0=15567282.39985 +y_0=336410.83237']

regions=['01','02','03','04','05','06','07','08','09','10','11','12']

for proj, region in zip(src_proj, regions):
    inrast=rpath+'ETOPO1_Region'+region+'_Shelf_1km'
    outrast=rpath+'ETOPO1_Region'+region+'_Shelf_1km_GCS'
    projection_command_string='gdalwarp '+inrast+' '+outrast+' '+'-s_srs '+'\"'+proj+'\"'+' '+'-t_srs \"+proj=longlat +ellps=WGS84\"'
    os.system(projection_command_string)
    #os.system('gdalwarp %s %s -s_srs %s -t_srs +proj=longlat +ellps=WGS84' %(inrast, outrast, proj) )

print('All done')

All done
